# 主成分分析とk-meansによる画像クラスタリング

1. データ読み込み
2. データの理解
3. 主成分分析
4. k-means
5. 可視化

In [1]:
# 必要なモジュールの読み込み
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt

In [5]:
# 今回使う顔画像データのダウンロード
people = fetch_lfw_people(min_faces_per_person=20,resize=0.7)

ImportError: The Python Imaging Library (PIL) is required to load data from jpeg files

In [ ]:
dir(people)　# ['DESCR', 'data', 'images', 'target', 'target_names']

# 顔データセットの確認
print(people.DESCR)

データの確認

In [ ]:
print( people.data ）
print( people.data.shape ) # (3023, 5655)
print( people.target.shape ) # (3023,)
print( people.target_names.shape ) # (62,)
print( people.images.shape ) # (3023, 87, 65)
print( people.images[0].shape )　# (87, 65)

データの可視化

In [ ]:
image_shape = people.images[0].shape
# subplots()にて、２行5列のデータとして可視化
fix, axis = plt.subplots(2, 5, figsize=(15, 8), subplot_kw={'xticks':(), 'yticks':()})
for target, image, ax in zip(people.target, people.images, axis.ravel()): #axis.ravelで一次元配列に
    ax.imshow(image) # 画像の表示
    ax.set_title(people.target_names[target]) # ラベル付け

In [ ]:
import numpy as np
np.bincount(people.target)
counts = np.bincount(people.target)

# それぞれの人のtarget_namesの数をカウントしたデータを出力
for i, (count, name) in enumerate(zip(counts, people.target_names)):
    print("{0:25} {1:3}".format(name, count), end='   ')
    if (i+1) % 3 == 0:
        print()

"""
Alejandro Toledo           39   Alvaro Uribe               35   Amelie Mauresmo            21   
Andre Agassi               36   Angelina Jolie             20   Ariel Sharon               77   
Arnold Schwarzenegger      42   Atal Bihari Vajpayee       24   Bill Clinton               29   
Carlos Menem               21   Colin Powell              236   David Beckham              31   
Donald Rumsfeld           121   George Robertson           22   George W Bush             530   
Gerhard Schroeder         109   Gloria Macapagal Arroyo    44   Gray Davis                 26   
Guillermo Coria            30   Hamid Karzai               22   Hans Blix                  39   
Hugo Chavez                71   Igor Ivanov                20   Jack Straw                 28   
Jacques Chirac             52   Jean Chretien              55   Jennifer Aniston           21   
Jennifer Capriati          42   Jennifer Lopez             21   Jeremy Greenstock          24   
Jiang Zemin                20   John Ashcroft              53   John Negroponte            31   
Jose Maria Aznar           23   Juan Carlos Ferrero        28   Junichiro Koizumi          60   
Kofi Annan                 32   Laura Bush                 41   Lindsay Davenport          22   
Lleyton Hewitt             41   Luiz Inacio Lula da Silva  48   Mahmoud Abbas              29   
Megawati Sukarnoputri      33   Michael Bloomberg          20   Naomi Watts                22   
Nestor Kirchner            37   Paul Bremer                20   Pete Sampras               22   
Recep Tayyip Erdogan       30   Ricardo Lagos              27   Roh Moo-hyun               32   
Rudolph Giuliani           26   Saddam Hussein             23   Serena Williams            52   
Silvio Berlusconi          33   Tiger Woods                23   Tom Daschle                25   
Tom Ridge                  33   Tony Blair                144   Vicente Fox                32   
Vladimir Putin             49   Winona Ryder               24   
"""

主成分分析

In [ ]:
# 主成分分析(PCA)を読み込む
from sklearn.decomposition import PCA
mask = np.zeros(people.target.shape, dtype=np.bool) #targetの形と同じ形の(3023)のぜろ行列Bool値を入れてる=> 全てFalse

"""
上の結果を見ると、
Alejandro Toledoは39しかなく、
George W Bushは530 もあるので、
全体の偏りをなくすため、それぞれの最大値を50にしています。
"""

for target in np.unique(people.target): #重複したものを削除
       mask[np.where(people.target == target)[0][:50]] = 1 

X_people = people.data[mask]
y_people = people.target[mask]

X_people = X_people / 255

# 主成分分析
pca = PCA(n_components=50, whiten=True, random_state=0)
X_pca = pca.fit_transform(X_people)

K-means

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=10, random_state=0)
labels_km = km.fit_predict(X_pca)
print("クラスターサイズ:{}".format(np.bincount(labels_km)))

mglearnモジュールによる可視化

In [ ]:
import mglearn
# mglearnのplot_kmeans_faces()を使うことで、簡単に顔画像を表示出来ます。
mglearn.plots.plot_kmeans_faces(km, pca, X_pca, X_people, y_people, people.target_names)